This notebook, adapted from https://github.com/dhuynh95/homomorphic_encryption_intro.

Please visit the original notebook for further details.

# Introduction to encoding in CKKS


<a href="https://colab.research.google.com/github/textualization/riiaa21_ws11_ml_over_encrypted_data/blob/main/notebooks/2_Encoding_decoding_ckks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


CKKS exploits the rich structure of integer polynomial rings for its plaintext and ciphertext spaces. Nonetheless, data comes more often in the form of vectors than polynomials.

Therefore it becomes necessary to encode our input $z \in \mathbb{C}^{N/2}$, into a polynomial $m(X) \in \mathbb{Z}[X]/(X^N + 1)$. 

We will denote by $N$ the degree of our polynomial degree modulus, which will be a power of 2. We denote by $\Phi_M(X) = X^N + 1$ the $m$-th [cyclotomic polynomial](https://en.wikipedia.org/wiki/Cyclotomic_polynomial) (note that $M=2N$). The plaintext space will be the polynomial ring $\mathcal{R} = \mathbb{Z}[X]/(X^N + 1)$. Let us denote by $\xi_M$, the $M$-th root of unity : $\xi_M = e^{2 i \pi / M}$.

To understand how we can encode a vector into a polynomial, and how the computation performed on the polynomial will be reflected in the underlying vector, we will first see a vanilla example, where we simply encode a vector $z \in \mathbb{C}^{N}$ into a polynomial $m(X) \in \mathbb{C}[X]/(X^N + 1)$. Then we will cover the actual encoding of CKKS, which takes a vector $z \in \mathbb{C}^{N/2}$, and encodes it in a polynomial $m(X) \in \mathbb{Z}[X]/(X^N + 1)$.

## Vanilla encoding

Here we will cover the simple case of encoding $z \in \mathbb{C}^{N}$, into a polynomial $m(X) \in \mathbb{C}[X]/(X^N + 1)$. 

To do so, we will use the [canonical embedding](https://en.wikipedia.org/wiki/Embedding) $\sigma : \mathbb{C}[X]/(X^N +1) \rightarrow \mathbb{C}^N$, which will serve to decode and encode our vectors.

The idea is simple, to decode a polynomial $m(X)$ into a vector $z$, we will simply evaluate the polynomial on certain values, which will be the roots of the cyclotomic polynomial $\Phi_M(X) = X^N + 1$. Those $N$ roots are : $\xi, \xi^3, ..., \xi^{2 N - 1}$. 

So to decode a polynomial $m(X)$, we will simply define $\sigma(m) = (m(\xi), m(\xi^3), ..., m(\xi^{2 N - 1})) \in \mathbb{C}^N$. Note that $\sigma$ defines an isomorphism, which means it is a bijective homomorphism, therefore any vector will be uniquely encoded into its corresponding polynomial, and vice-versa.

The tricky part is the encoding of a vector $z \in \mathbb{C}^N$ into the corresponding polynomial, which means computing the inverse $\sigma^{-1}$. The problem is therefore to find a polynomial $m(X) = \sum_{i=0}^{N-1} \alpha_i X^i \in \mathbb{C}[X]/(X^N + 1)$, given a vector $z \in \mathbb{C}^N$, such that $\sigma(m) = (m(\xi), m(\xi^3), ..., m(\xi^{2 N - 1})) = (z_1,...,z_N)$.

Nonetheless, if we write this down, this means that we have the following system : 

$\sum_{j=0}^{N-1} \alpha_j (\xi^{2 i - 1})^j = z_i, i=1,...,N$.

This can be viewed as a linear equation : 

$A \alpha = z$, with $A$ the [Vandermonde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix) of the $(\xi^{2 i -1})_{i=1,...,N}$, $\alpha$ the vector of the polynomial coefficients, and $z$ the vector we want to encode.

Therefore we have that : $\alpha = A^{-1} z$, and that $\sigma^{-1}(z) = \sum_{i=0}^{N-1} \alpha_i X^i \in \mathbb{C}[X]/(X^N + 1)$.

### Example

We will see an example now to better understand what this means.

Let $M = 8$, $N = \frac{M}{2}= 4$, $\Phi_M(X) = X^4 + 1$, and $\omega = e^{\frac{2 i \pi}{8}} = e^{\frac{i \pi}{4}}$.
Our goal here is to encode the following vectors : $[1, 2, 3, 4]$ and $[-1, -2, -3, -4]$, decode them, add and multiply their polynomial and decode it.

As we saw, in order to decode a polynomial, we simply need to evaluate it on powers of an $M$-th root of unity. Here we choose $\xi_M = \omega = e^{\frac{i \pi}{4}}$.

Once we have $\xi$ and $M$, we can define both $\sigma$ and its inverse $\sigma^{-1}$, respectively the decoding and the encoding.

In [1]:
import numpy as np

# First we set the parameters
M = 8
N = M //2

# We set xi, which will be used in our computations
xi = np.exp(2 * np.pi * 1j / M)
xi

(0.7071067811865476+0.7071067811865475j)

In [2]:
from numpy.polynomial import Polynomial

class CKKSEncoder:
    """Basic CKKS encoder to encode complex vectors into polynomials."""
    
    def __init__(self, M: int):
        """Initialization of the encoder for M a power of 2. 
        
        xi, which is an M-th root of unity will, be used as a basis for our computations.
        """
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M
        
    @staticmethod
    def vandermonde(xi: np.complex128, M: int) -> np.array:
        """Computes the Vandermonde matrix from a m-th root of unity."""
        
        N = M //2
        matrix = []
        # We will generate each row of the matrix
        for i in range(N):
            # For each row we select a different root
            root = xi ** (2 * i + 1)
            row = []

            # Then we store its powers
            for j in range(N):
                row.append(root ** j)
            matrix.append(row)
        return matrix
    
    def sigma_inverse(self, b: np.array) -> Polynomial:
        """Encodes the vector b in a polynomial using an M-th root of unity."""

        # First we create the Vandermonde matrix
        A = CKKSEncoder.vandermonde(self.xi, M)

        # Then we solve the system
        coeffs = np.linalg.solve(A, b)

        # Finally we output the polynomial
        p = Polynomial(coeffs)
        return p

    def sigma(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by applying it to the M-th roots of unity."""

        outputs = []
        N = self.M //2

        # We simply apply the polynomial on the roots
        for i in range(N):
            root = self.xi ** (2 * i + 1)
            output = p(root)
            outputs.append(output)
        return np.array(outputs)

Now we can start experimenting with real values, let's first encode a vector and see how it is encoded.

In [3]:
# First we initialize our encoder
encoder = CKKSEncoder(M)

In [4]:
b = np.array([1, 2, 3, 4])
b

array([1, 2, 3, 4])

Let's encode the vector now.

In [5]:
p = encoder.sigma_inverse(b)
p

Polynomial([ 2.50000000e+00+4.44089210e-16j, -4.99600361e-16+7.07106781e-01j,
       -3.46944695e-16+5.00000000e-01j, -8.32667268e-16+7.07106781e-01j], domain=[-1,  1], window=[-1,  1])

Let's see now how we can extract the vector we had initially from the polynomial: 

In [6]:
b_reconstructed = encoder.sigma(p)
b_reconstructed

array([1.-1.11022302e-16j, 2.-4.71844785e-16j, 3.+2.77555756e-17j,
       4.+2.22044605e-16j])

We can see that the reconstruction and the initial vector are very close.

In [7]:
np.linalg.norm(b_reconstructed - b)

6.944442800358888e-16

As stated before, $\sigma$ is not chosen randomly to encode and decode, but has a lot of nice properties. Among them, $\sigma$ is an isomorphism, therefore addition and multiplication on polynomials will result in coefficient wise addition and multiplication on the encoded vectors.

The homomorphic property of $\sigma$ is due to the fact that $X^N = 1$ and $\xi^N = 1$.

We can now start to encode several vectors, and see how we can perform homomorphic operations on them and decode it.

In [8]:
m1 = np.array([1, 2, 3, 4])
m2 = np.array([1, -2, 3, -4])

In [9]:
p1 = encoder.sigma_inverse(m1)
p2 = encoder.sigma_inverse(m2)

We can see that addition is pretty straightforward.

In [10]:
p_add = p1 + p2
p_add

Polynomial([ 2.00000000e+00+2.22044605e-16j, -7.07106781e-01+7.07106781e-01j,
        2.10942375e-15-2.00000000e+00j,  7.07106781e-01+7.07106781e-01j], domain=[-1.,  1.], window=[-1.,  1.])

Here as expected, we see that p1 + p2 decodes correctly to $[2, 0, 6, 0]$.

In [11]:
encoder.sigma(p_add)

array([2.0000000e+00+1.43539982e-16j, 4.4408921e-16-3.33066907e-16j,
       6.0000000e+00+2.22044605e-16j, 4.4408921e-16+4.44089210e-16j])

Because when doing multiplication we might have terms whose degree is higher than $N$, we will need to do a modulo operation using $X^N + 1$.

To perform multiplication, we first need to define the polynomial modulus which we will use.

In [12]:
poly_modulo = Polynomial([1,0,0,0,1])
poly_modulo

Polynomial([1., 0., 0., 0., 1.], domain=[-1,  1], window=[-1,  1])

Now we can perform multiplication.

In [13]:
p_mult = p1 * p2 % poly_modulo

Finally if we decode it, we can see that we have the expected result.

In [14]:
encoder.sigma(p_mult)

array([  1.-6.03897574e-16j,  -4.+9.50414660e-16j,   9.+9.50414660e-16j,
       -16.-8.81954796e-15j])

## CKKS encoding

Now that we saw how we can encode complex vectors into polynomials, let's see how it is done in CKKS.

The difference with what we did previously, is that the plaintext space of the encoded polynomial is $\mathcal{R} = \mathbb{Z}[X]/(X^N + 1)$, therefore the coefficients of the polynomial of encoded values must have integer coefficients, and we saw before that it is not neccessarily the case.

To solve this issue, one must first look at the image of the canonical embedding $\sigma$ on $\mathcal{R}$.

Because polynomials in $\mathcal{R}$ have integer coefficients, therefore real coefficients, and we evaluate them on complex roots, where half are the conjugates of the other (see the previous figure), we have that :
$\sigma(\mathcal{R}) \subseteq \mathbb{H} = \{z \in \mathbb{C}^{N} : z_j = \overline{z_{-j}} \}$. 

Any element of $\sigma(\mathcal{R})$ is actually in a space of dimension $N/2$, not $N$. Therefore when we want to encode a vector in CKKS we use complex vectors of size $N/2$, then we need to expand them by copying the other half of conjugate roots.

This operation, which takes an element of $\mathbb{H}$ and projects it to $\mathbb{C}^{N/2}$ is called $\pi$ in the CKKS paper. Note that this defines a isomorphism as well. 

So now we can start with $z \in \mathbb{C}^{N/2}$, expand it using $\pi^{-1}$ (note that $\pi$ projects, $\pi^{-1}$ expands), so we get $\pi^{-1}(z) \in \mathbb{H}$.

The problem that we have is that we cannot directly use $\sigma : \mathcal{R} = \mathbb{Z}[X]/(X^N +1) \rightarrow \sigma(\mathcal{R}) \subseteq \mathbb{H}$, because an element of $\mathbb{H}$ is not necessarily in $\sigma(\mathcal{R})$. $\sigma$ does define an isomorphism, but only from $\mathcal{R}$ to $\sigma(\mathcal{R})$. To convince yourself that $\sigma(\mathcal{R})$ is not equal to $\mathbb{H}$, you can notice that $\mathcal{R}$ is countable, therefore $\sigma(\mathcal{R})$ as well, but $\mathbb{H}$ is not, as it is isomorph to $\mathbb{C}^{N/2}$.

This detail is important, because it means that we must find a way to project $\pi^{-1}(z)$ on $\sigma(\mathcal{R})$. To do so, we will use a technique called "coordinate-wise random rounding", defined in [A Toolkit for Ring-LWE Cryptography](https://web.eecs.umich.edu/~cpeikert/pubs/toolkit.pdf). 

The idea is simple, $\mathcal{R}$ has an orthogonal $\mathbb{Z}$-basis $\{1,X,...,X^{N-1} \}$, and given that $\sigma$ is an isomorphism, $\sigma(\mathcal{R})$ has an orthogonal $\mathbb{Z}$-basis $\beta = (b_1,b_2,...,b_N) = (\sigma(1),\sigma(X),...,\sigma(X^{N-1}) )$. Therefore for any $z \in \mathbb{H}$, we will simply project it on $\beta$ :

$z = \sum_{i=1}^{N} z_i b_i$, with $z_i = \frac{<z, b_i>}{||bi||^2} \in \mathbb{R}$ because the basis is orthogonal and not orthonormal. Note that we are using the hermitian product here : $<x,y> = \sum_{i=1}^{N} x_i \overline{y_i}$. Here the hermitian product gives real outputs because we apply it on elements of $\mathbb{H}$, you can compute to convince yourself, or notice that you can find an isometric isomorphism between $\mathbb{H}$ and $\mathbb{R}^N$, therefore inner product in $\mathbb{H}$ will yield real output.

Finally once we have the $z_i$, we simply need to round them randomly, to the higher or the lower integer, using the "coordinate-wise random rounding".

Once we have projected on $\sigma(\mathcal{R})$, we can simply apply $\sigma^{-1}$ which will output an element of $\mathcal{R}$ which was what we wanted ! 

One final detail : because the rounding might destroy some significant numbers, we actually need to multply by $\Delta > 0$ which gives us some precision. 

So the final encoding procedure is : 
- take an element of $z \in \mathcal{C}^{N/2}$
- expand it to $\pi^{-1}(z) \in \mathbb{H}$
- multiply it by $\Delta$ for precision
- project it on $\sigma(\mathcal{R})$ : $\lfloor \Delta . \pi^{-1}(z) \rceil_{\sigma(\mathcal{R})} \in \sigma(\mathcal{R})$
- encode it using $\sigma$ : $m(X) = \sigma^{-1} (\lfloor \Delta . \pi^{-1}(z) \rceil_{\sigma(\mathbb{R})}) \in \mathcal{R}$.

The decoding procedure is much simpler, from a polynomial $m(X)$ we simply get $z = \pi \circ \sigma(\Delta^{-1} . m)$

Pfiou that was long, but that was all the math, now it will be pretty straightforward to implement in the code so let's do it ! 

## Example

Here for the rest of the notebook we choose to keep building upon the `CKKSEncoder` class we have defined earlier. Instead of redefining the class each time we want to add or change methods, we will simply use `patch_to` from the `fastcore` package from [Fastai](https://github.com/fastai/fastai). This allows to monkey patch objects that have already been defined. This is purely for conveniency, and you could just redefine the `CKKSEncoder` at each cell with the added methods.

In [15]:
!pip3 install fastcore

from fastcore.foundation import patch_to

In [16]:
@patch_to(CKKSEncoder)
def pi(self, z: np.array) -> np.array:
    """Projects a vector of H into C^{N/2}."""
    
    N = self.M // 4
    return z[:N]

@patch_to(CKKSEncoder)
def pi_inverse(self, z: np.array) -> np.array:
    """Expands a vector of C^{N/2} by expanding it with its
    complex conjugate."""
    
    z_conjugate = z[::-1]
    z_conjugate = [np.conjugate(x) for x in z_conjugate]
    return np.concatenate([z, z_conjugate])

# We can now initialize our encoder with the added methods
encoder = CKKSEncoder(M)

In [17]:
z = np.array([0,1])

In [18]:
encoder.pi_inverse(z)

array([0, 1, 1, 0])

In [19]:
@patch_to(CKKSEncoder)
def create_sigma_R_basis(self):
    """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""

    self.sigma_R_basis = np.array(self.vandermonde(self.xi, self.M)).T
    
@patch_to(CKKSEncoder)
def __init__(self, M):
    """Initialize with the basis"""
    self.xi = np.exp(2 * np.pi * 1j / M)
    self.M = M
    self.create_sigma_R_basis()
    
encoder = CKKSEncoder(M)

We can now have a look at the basis $\sigma(1), \sigma(X), \sigma(X^2), \sigma(X^3)$.

In [20]:
encoder.sigma_R_basis

array([[ 1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ,
         1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ],
       [ 7.07106781e-01+0.70710678j, -7.07106781e-01+0.70710678j,
        -7.07106781e-01-0.70710678j,  7.07106781e-01-0.70710678j],
       [ 2.22044605e-16+1.j        , -4.44089210e-16-1.j        ,
         1.11022302e-15+1.j        , -1.38777878e-15-1.j        ],
       [-7.07106781e-01+0.70710678j,  7.07106781e-01+0.70710678j,
         7.07106781e-01-0.70710678j, -7.07106781e-01-0.70710678j]])

Here we will check that elements of $\mathbb{Z} \{ \sigma(1), \sigma(X), \sigma(X^2), \sigma(X^3) \}$ are encoded as integer polynomials.

In [21]:
# Here we simply take a vector whose coordinates are (1,1,1,1) in the lattice basis
coordinates = [1,1,1,1]

b = np.matmul(encoder.sigma_R_basis.T, coordinates)
b

array([1.+2.41421356j, 1.+0.41421356j, 1.-0.41421356j, 1.-2.41421356j])

We can check now that it does encode to an integer polynomial.

In [22]:
p = encoder.sigma_inverse(b)
p

Polynomial([1.+2.22044605e-16j, 1.+0.00000000e+00j, 1.+2.77555756e-17j,
       1.+2.22044605e-16j], domain=[-1,  1], window=[-1,  1])

In [23]:
@patch_to(CKKSEncoder)
def compute_basis_coordinates(self, z):
    """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
    output = np.array([np.real(np.vdot(z, b) / np.vdot(b,b)) for b in self.sigma_R_basis])
    return output

def round_coordinates(coordinates):
    """Gives the integral rest."""
    coordinates = coordinates - np.floor(coordinates)
    return coordinates

def coordinate_wise_random_rounding(coordinates):
    """Rounds coordinates randonmly."""
    r = round_coordinates(coordinates)
    f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)
    
    rounded_coordinates = coordinates - f
    rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
    return rounded_coordinates

@patch_to(CKKSEncoder)
def sigma_R_discretization(self, z):
    """Projects a vector on the lattice using coordinate wise random rounding."""
    coordinates = self.compute_basis_coordinates(z)
    
    rounded_coordinates = coordinate_wise_random_rounding(coordinates)
    y = np.matmul(self.sigma_R_basis.T, rounded_coordinates)
    return y

encoder = CKKSEncoder(M)

Finally, because there might be loss of precisions during the rounding step, we had the scale parameter $\delta$, to allow a fixed level of precision.

In [24]:
@patch_to(CKKSEncoder)
def __init__(self, M:int, scale:float):
    """Initializes with scale."""
    self.xi = np.exp(2 * np.pi * 1j / M)
    self.M = M
    self.create_sigma_R_basis()
    self.scale = scale
    
@patch_to(CKKSEncoder)
def encode(self, z: np.array) -> Polynomial:
    """Encodes a vector by expanding it first to H,
    scale it, project it on the lattice of sigma(R), and performs
    sigma inverse.
    """
    pi_z = self.pi_inverse(z)
    scaled_pi_z = self.scale * pi_z
    rounded_scale_pi_zi = self.sigma_R_discretization(scaled_pi_z)
    p = self.sigma_inverse(rounded_scale_pi_zi)
    
    # We round it afterwards due to numerical imprecision
    coef = np.round(np.real(p.coef)).astype(int)
    p = Polynomial(coef)
    return p

@patch_to(CKKSEncoder)
def decode(self, p: Polynomial) -> np.array:
    """Decodes a polynomial by removing the scale, 
    evaluating on the roots, and project it on C^(N/2)"""
    rescaled_p = p / self.scale
    z = self.sigma(rescaled_p)
    pi_z = self.pi(z)
    return pi_z

scale = 64

encoder = CKKSEncoder(M, scale)

We can now see it on action, the full encoder used by CKKS : 

In [25]:
z = np.array([3 +4j, 2 - 1j])
z

array([3.+4.j, 2.-1.j])

We can see that we now have an integer polynomial as our encoding.

In [26]:
p = encoder.encode(z)
p

Polynomial([160.,  91., 160.,  45.], domain=[-1,  1], window=[-1,  1])

And it actually decodes well ! 

In [27]:
encoder.decode(p)

array([3.008233+4.00260191j, 1.991767-0.99739809j])